In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import glob
import matplotlib.pyplot as plt
from keras import layers
from keras import Model
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
import os
from mlxtend.data import loadlocal_mnist

# load training data
X, y = loadlocal_mnist(
    images_path='../emnist/emnist-letters-train-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-train-labels-idx1-ubyte')

X = X.reshape(124800, 28, 28, 1)
X = np.divide(X, 255.)
y = y-1

# load testing data
Xtest, ytest = loadlocal_mnist(
    images_path='../emnist/emnist-letters-test-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-test-labels-idx1-ubyte')

Xtest = Xtest.reshape(20800, 28, 28, 1)
Xtest = np.divide(Xtest, 255.)
ytest = ytest-1

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12408717754414148642
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 23285923840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16548739099426714998
 physical_device_desc: "device: 0, name: Tesla M40 24GB, pci bus id: 0000:01:00.0, compute capability: 5.2"
 xla_global_id: 416903419]

## Model 1: Same architecture as in-class CNN example

In [7]:
np.random.seed(4622)
tf.random.set_seed(4622)

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B

img_input = layers.Input(shape=(28,28,1))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(16,3,activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(32,3,activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window

x = layers.Conv2D(64,3,activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers

x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units

x = layers.Dense(512, activation='relu')(x) # captures more non-neighbor and non-linear relationships

# Create output layer with a single node and sigmoid activation

output = layers.Dense(26, activation='softmax')(x)

# Create model:

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

In [8]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 2s 558ms/step - loss: 3.2494 - accuracy: 0.0659 - val_loss: 3.2314 - val_accuracy: 0.0837
Epoch 2/1000
4/4 [==============================] - 1s 300ms/step - loss: 3.2151 - accuracy: 0.1145 - val_loss: 3.1808 - val_accuracy: 0.1768
Epoch 3/1000
4/4 [==============================] - 1s 304ms/step - loss: 3.1482 - accuracy: 0.2107 - val_loss: 3.0814 - val_accuracy: 0.2648
Epoch 4/1000
4/4 [==============================] - 1s 304ms/step - loss: 3.0200 - accuracy: 0.3131 - val_loss: 2.8992 - val_accuracy: 0.3716
Epoch 5/1000
4/4 [==============================] - 1s 304ms/step - loss: 2.7978 - accuracy: 0.3820 - val_loss: 2.6136 - val_accuracy: 0.3934
Epoch 6/1000
4/4 [==============================] - 1s 299ms/step - loss: 2.4859 - accuracy: 0.4118 - val_loss: 2.2827 - val_accuracy: 0.4375
Epoch 7/1000
4/4 [==============================] - 1s 302ms/step - loss: 2.1667 - accuracy: 0.4451 - val_loss: 2.0019 - val_accuracy: 0.4605
Epoch 

4/4 [==============================] - 1s 305ms/step - loss: 0.5480 - accuracy: 0.8217 - val_loss: 0.5449 - val_accuracy: 0.8243
Epoch 59/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.5415 - accuracy: 0.8244 - val_loss: 0.5367 - val_accuracy: 0.8248
Epoch 60/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.5360 - accuracy: 0.8267 - val_loss: 0.5351 - val_accuracy: 0.8261
Epoch 61/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.5329 - accuracy: 0.8271 - val_loss: 0.5299 - val_accuracy: 0.8259
Epoch 62/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.5295 - accuracy: 0.8284 - val_loss: 0.5257 - val_accuracy: 0.8280
Epoch 63/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.5250 - accuracy: 0.8293 - val_loss: 0.5221 - val_accuracy: 0.8299
Epoch 64/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.5192 - accuracy: 0.8315 - val_loss: 0.5179 - val_accuracy: 0.8307
Epoch 65/1000

4/4 [==============================] - 1s 301ms/step - loss: 0.3901 - accuracy: 0.8715 - val_loss: 0.4047 - val_accuracy: 0.8697
Epoch 116/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.3883 - accuracy: 0.8718 - val_loss: 0.4095 - val_accuracy: 0.8677
Epoch 117/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3888 - accuracy: 0.8723 - val_loss: 0.4061 - val_accuracy: 0.8692
Epoch 118/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.3864 - accuracy: 0.8733 - val_loss: 0.4044 - val_accuracy: 0.8695
Epoch 119/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3863 - accuracy: 0.8732 - val_loss: 0.4018 - val_accuracy: 0.8702
Epoch 120/1000
4/4 [==============================] - 1s 323ms/step - loss: 0.3837 - accuracy: 0.8733 - val_loss: 0.4045 - val_accuracy: 0.8698
Epoch 121/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.3869 - accuracy: 0.8713 - val_loss: 0.4012 - val_accuracy: 0.8722
Epoch 1

Epoch 172/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.3321 - accuracy: 0.8893 - val_loss: 0.3644 - val_accuracy: 0.8820
Epoch 173/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3298 - accuracy: 0.8903 - val_loss: 0.3600 - val_accuracy: 0.8849
Epoch 174/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.3284 - accuracy: 0.8909 - val_loss: 0.3606 - val_accuracy: 0.8831
Epoch 175/1000
4/4 [==============================] - 1s 306ms/step - loss: 0.3280 - accuracy: 0.8909 - val_loss: 0.3580 - val_accuracy: 0.8845
Epoch 176/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.3256 - accuracy: 0.8919 - val_loss: 0.3558 - val_accuracy: 0.8860
Epoch 177/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.3256 - accuracy: 0.8913 - val_loss: 0.3579 - val_accuracy: 0.8846
Epoch 178/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.3254 - accuracy: 0.8920 - val_loss: 0.3559 - val_accuracy:

Epoch 229/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.2958 - accuracy: 0.9004 - val_loss: 0.3352 - val_accuracy: 0.8915
Epoch 230/1000
4/4 [==============================] - 1s 309ms/step - loss: 0.2949 - accuracy: 0.9010 - val_loss: 0.3343 - val_accuracy: 0.8923
Epoch 231/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.2939 - accuracy: 0.9015 - val_loss: 0.3376 - val_accuracy: 0.8914
Epoch 232/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.2940 - accuracy: 0.9010 - val_loss: 0.3359 - val_accuracy: 0.8921
Epoch 233/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.2923 - accuracy: 0.9021 - val_loss: 0.3363 - val_accuracy: 0.8913
Epoch 234/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.2929 - accuracy: 0.9016 - val_loss: 0.3380 - val_accuracy: 0.8903
Epoch 235/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.2933 - accuracy: 0.9014 - val_loss: 0.3349 - val_accuracy:

...

Epoch 742/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1820 - accuracy: 0.9362 - val_loss: 0.3034 - val_accuracy: 0.9032
Epoch 743/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1819 - accuracy: 0.9369 - val_loss: 0.3044 - val_accuracy: 0.9033
Epoch 744/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.1810 - accuracy: 0.9372 - val_loss: 0.3022 - val_accuracy: 0.9044
Epoch 745/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1808 - accuracy: 0.9369 - val_loss: 0.3078 - val_accuracy: 0.9040
Epoch 746/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1832 - accuracy: 0.9361 - val_loss: 0.3038 - val_accuracy: 0.9025
Epoch 747/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1812 - accuracy: 0.9368 - val_loss: 0.3019 - val_accuracy: 0.9042
Epoch 748/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.1802 - accuracy: 0.9370 - val_loss: 0.3085 - val_accuracy:

Epoch 799/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1774 - accuracy: 0.9378 - val_loss: 0.3128 - val_accuracy: 0.9015
Epoch 800/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.1808 - accuracy: 0.9358 - val_loss: 0.3119 - val_accuracy: 0.9030
Epoch 801/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.1782 - accuracy: 0.9370 - val_loss: 0.3062 - val_accuracy: 0.9029
Epoch 802/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1767 - accuracy: 0.9383 - val_loss: 0.3124 - val_accuracy: 0.9017
Epoch 803/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1783 - accuracy: 0.9369 - val_loss: 0.3081 - val_accuracy: 0.9042
Epoch 804/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1763 - accuracy: 0.9377 - val_loss: 0.3084 - val_accuracy: 0.9037
Epoch 805/1000
4/4 [==============================] - 1s 308ms/step - loss: 0.1766 - accuracy: 0.9375 - val_loss: 0.3142 - val_accuracy:

Epoch 856/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1693 - accuracy: 0.9405 - val_loss: 0.3080 - val_accuracy: 0.9049
Epoch 857/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1688 - accuracy: 0.9410 - val_loss: 0.3073 - val_accuracy: 0.9050
Epoch 858/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.1675 - accuracy: 0.9416 - val_loss: 0.3069 - val_accuracy: 0.9043
Epoch 859/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1667 - accuracy: 0.9419 - val_loss: 0.3068 - val_accuracy: 0.9056
Epoch 860/1000
4/4 [==============================] - 1s 309ms/step - loss: 0.1679 - accuracy: 0.9414 - val_loss: 0.3094 - val_accuracy: 0.9047
Epoch 861/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1709 - accuracy: 0.9395 - val_loss: 0.3116 - val_accuracy: 0.9024
Epoch 862/1000
4/4 [==============================] - 1s 303ms/step - loss: 0.1709 - accuracy: 0.9392 - val_loss: 0.3129 - val_accuracy:

Epoch 913/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1636 - accuracy: 0.9418 - val_loss: 0.3096 - val_accuracy: 0.9040
Epoch 914/1000
4/4 [==============================] - 1s 305ms/step - loss: 0.1636 - accuracy: 0.9424 - val_loss: 0.3150 - val_accuracy: 0.9036
Epoch 915/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1639 - accuracy: 0.9422 - val_loss: 0.3088 - val_accuracy: 0.9053
Epoch 916/1000
4/4 [==============================] - 1s 304ms/step - loss: 0.1623 - accuracy: 0.9427 - val_loss: 0.3110 - val_accuracy: 0.9032
Epoch 917/1000
4/4 [==============================] - 1s 310ms/step - loss: 0.1639 - accuracy: 0.9419 - val_loss: 0.3097 - val_accuracy: 0.9054
Epoch 918/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1629 - accuracy: 0.9428 - val_loss: 0.3092 - val_accuracy: 0.9048
Epoch 919/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.1624 - accuracy: 0.9428 - val_loss: 0.3090 - val_accuracy:

Epoch 970/1000
4/4 [==============================] - 1s 299ms/step - loss: 0.1560 - accuracy: 0.9449 - val_loss: 0.3121 - val_accuracy: 0.9049
Epoch 971/1000
4/4 [==============================] - 1s 301ms/step - loss: 0.1559 - accuracy: 0.9453 - val_loss: 0.3142 - val_accuracy: 0.9026
Epoch 972/1000
4/4 [==============================] - 1s 307ms/step - loss: 0.1571 - accuracy: 0.9448 - val_loss: 0.3164 - val_accuracy: 0.9038
Epoch 973/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1568 - accuracy: 0.9449 - val_loss: 0.3113 - val_accuracy: 0.9051
Epoch 974/1000
4/4 [==============================] - 1s 309ms/step - loss: 0.1548 - accuracy: 0.9459 - val_loss: 0.3109 - val_accuracy: 0.9058
Epoch 975/1000
4/4 [==============================] - 1s 302ms/step - loss: 0.1553 - accuracy: 0.9459 - val_loss: 0.3130 - val_accuracy: 0.9041
Epoch 976/1000
4/4 [==============================] - 1s 300ms/step - loss: 0.1551 - accuracy: 0.9459 - val_loss: 0.3165 - val_accuracy:

In [9]:
model.save("../models/cnn1")

np.save('../models/cnn1_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn1_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn1_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn1_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn1/assets


## Model 1 Testing

In [27]:
Xtest, ytest = loadlocal_mnist(
    images_path='../emnist/emnist-letters-test-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-test-labels-idx1-ubyte')

Xtest = Xtest.reshape(20800, 28, 28, 1)
Xtest = np.divide(Xtest, 255.)
ytest = ytest-1

In [33]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

Accuracy is 0.8902403846153846
F1 score is 0.8903074237471855


## Model 2: Going deeper with dropout

In [4]:
np.random.seed(4622)
tf.random.set_seed(4622)

img_input = layers.Input(shape=(28,28,1))

x = layers.Conv2D(16, 3, activation='relu', padding='same')(img_input)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(32, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.4)(x)

output = layers.Dense(26, activation='softmax')(x)

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss, metrics=['accuracy'])

In [5]:
history = model.fit(X, y, epochs=1000, batch_size=31200, validation_data=(Xtest, ytest))

Epoch 1/1000
4/4 [==============================] - 33s 5s/step - loss: 3.2485 - accuracy: 0.0598 - val_loss: 3.2273 - val_accuracy: 0.1701
Epoch 2/1000
4/4 [==============================] - 13s 3s/step - loss: 2.9881 - accuracy: 0.1521 - val_loss: 2.7743 - val_accuracy: 0.1840
Epoch 3/1000
4/4 [==============================] - 13s 3s/step - loss: 2.9473 - accuracy: 0.1846 - val_loss: 2.8863 - val_accuracy: 0.2432
Epoch 4/1000
4/4 [==============================] - 13s 3s/step - loss: 2.7257 - accuracy: 0.2150 - val_loss: 2.6257 - val_accuracy: 0.3012
Epoch 5/1000
4/4 [==============================] - 13s 3s/step - loss: 2.2798 - accuracy: 0.3210 - val_loss: 1.8882 - val_accuracy: 0.4648
Epoch 6/1000
4/4 [==============================] - 13s 3s/step - loss: 1.9536 - accuracy: 0.4163 - val_loss: 1.7177 - val_accuracy: 0.5203
Epoch 7/1000
4/4 [==============================] - 13s 3s/step - loss: 1.6784 - accuracy: 0.4883 - val_loss: 1.4731 - val_accuracy: 0.5784
Epoch 8/1000
4/4 [==

4/4 [==============================] - 13s 3s/step - loss: 0.1690 - accuracy: 0.9392 - val_loss: 0.1596 - val_accuracy: 0.9451
Epoch 60/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1679 - accuracy: 0.9402 - val_loss: 0.1584 - val_accuracy: 0.9454
Epoch 61/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1650 - accuracy: 0.9410 - val_loss: 0.1580 - val_accuracy: 0.9464
Epoch 62/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1630 - accuracy: 0.9416 - val_loss: 0.1579 - val_accuracy: 0.9457
Epoch 63/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1619 - accuracy: 0.9416 - val_loss: 0.1571 - val_accuracy: 0.9465
Epoch 64/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1609 - accuracy: 0.9424 - val_loss: 0.1565 - val_accuracy: 0.9468
Epoch 65/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1585 - accuracy: 0.9428 - val_loss: 0.1563 - val_accuracy: 0.9468
Epoch 66/1000
4/4 [========

4/4 [==============================] - 13s 3s/step - loss: 0.1104 - accuracy: 0.9568 - val_loss: 0.1530 - val_accuracy: 0.9489
Epoch 118/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1113 - accuracy: 0.9562 - val_loss: 0.1537 - val_accuracy: 0.9482
Epoch 119/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1105 - accuracy: 0.9565 - val_loss: 0.1528 - val_accuracy: 0.9491
Epoch 120/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1096 - accuracy: 0.9573 - val_loss: 0.1528 - val_accuracy: 0.9488
Epoch 121/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1095 - accuracy: 0.9573 - val_loss: 0.1536 - val_accuracy: 0.9491
Epoch 122/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1072 - accuracy: 0.9574 - val_loss: 0.1546 - val_accuracy: 0.9486
Epoch 123/1000
4/4 [==============================] - 13s 3s/step - loss: 0.1085 - accuracy: 0.9571 - val_loss: 0.1536 - val_accuracy: 0.9482
Epoch 124/1000
4/4 [=

Epoch 175/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0855 - accuracy: 0.9646 - val_loss: 0.1649 - val_accuracy: 0.9484
Epoch 176/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0845 - accuracy: 0.9646 - val_loss: 0.1677 - val_accuracy: 0.9477
Epoch 177/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0853 - accuracy: 0.9646 - val_loss: 0.1652 - val_accuracy: 0.9482
Epoch 178/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0843 - accuracy: 0.9650 - val_loss: 0.1673 - val_accuracy: 0.9478
Epoch 179/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0851 - accuracy: 0.9647 - val_loss: 0.1640 - val_accuracy: 0.9491
Epoch 180/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0827 - accuracy: 0.9653 - val_loss: 0.1679 - val_accuracy: 0.9487
Epoch 181/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0826 - accuracy: 0.9654 - val_loss: 0.1661 - val_accuracy: 0.9491
Epoch 

Epoch 233/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0697 - accuracy: 0.9701 - val_loss: 0.1786 - val_accuracy: 0.9474
Epoch 234/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0713 - accuracy: 0.9694 - val_loss: 0.1791 - val_accuracy: 0.9475
Epoch 235/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0707 - accuracy: 0.9697 - val_loss: 0.1787 - val_accuracy: 0.9480
Epoch 236/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0697 - accuracy: 0.9704 - val_loss: 0.1778 - val_accuracy: 0.9474
Epoch 237/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0698 - accuracy: 0.9701 - val_loss: 0.1784 - val_accuracy: 0.9484
Epoch 238/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0700 - accuracy: 0.9701 - val_loss: 0.1785 - val_accuracy: 0.9477
Epoch 239/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0704 - accuracy: 0.9699 - val_loss: 0.1797 - val_accuracy: 0.9479
Epoch 

...

Epoch 755/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0340 - accuracy: 0.9855 - val_loss: 0.2505 - val_accuracy: 0.9452
Epoch 756/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0350 - accuracy: 0.9852 - val_loss: 0.2529 - val_accuracy: 0.9451
Epoch 757/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0350 - accuracy: 0.9849 - val_loss: 0.2502 - val_accuracy: 0.9453
Epoch 758/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0340 - accuracy: 0.9852 - val_loss: 0.2518 - val_accuracy: 0.9464
Epoch 759/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0343 - accuracy: 0.9848 - val_loss: 0.2522 - val_accuracy: 0.9458
Epoch 760/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0338 - accuracy: 0.9853 - val_loss: 0.2564 - val_accuracy: 0.9451
Epoch 761/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0340 - accuracy: 0.9852 - val_loss: 0.2583 - val_accuracy: 0.9454
Epoch 

Epoch 813/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0328 - accuracy: 0.9859 - val_loss: 0.2590 - val_accuracy: 0.9451
Epoch 814/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0337 - accuracy: 0.9857 - val_loss: 0.2531 - val_accuracy: 0.9458
Epoch 815/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0332 - accuracy: 0.9859 - val_loss: 0.2540 - val_accuracy: 0.9453
Epoch 816/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0335 - accuracy: 0.9854 - val_loss: 0.2540 - val_accuracy: 0.9450
Epoch 817/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0318 - accuracy: 0.9863 - val_loss: 0.2581 - val_accuracy: 0.9451
Epoch 818/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0327 - accuracy: 0.9861 - val_loss: 0.2573 - val_accuracy: 0.9451
Epoch 819/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0325 - accuracy: 0.9860 - val_loss: 0.2585 - val_accuracy: 0.9440
Epoch 

Epoch 871/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0305 - accuracy: 0.9867 - val_loss: 0.2604 - val_accuracy: 0.9450
Epoch 872/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0318 - accuracy: 0.9866 - val_loss: 0.2615 - val_accuracy: 0.9448
Epoch 873/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0311 - accuracy: 0.9868 - val_loss: 0.2631 - val_accuracy: 0.9447
Epoch 874/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0310 - accuracy: 0.9870 - val_loss: 0.2641 - val_accuracy: 0.9451
Epoch 875/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0301 - accuracy: 0.9872 - val_loss: 0.2621 - val_accuracy: 0.9457
Epoch 876/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0313 - accuracy: 0.9863 - val_loss: 0.2630 - val_accuracy: 0.9456
Epoch 877/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0312 - accuracy: 0.9870 - val_loss: 0.2626 - val_accuracy: 0.9450
Epoch 

Epoch 929/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0289 - accuracy: 0.9880 - val_loss: 0.2746 - val_accuracy: 0.9447
Epoch 930/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0290 - accuracy: 0.9879 - val_loss: 0.2711 - val_accuracy: 0.9449
Epoch 931/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0297 - accuracy: 0.9875 - val_loss: 0.2654 - val_accuracy: 0.9440
Epoch 932/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0298 - accuracy: 0.9872 - val_loss: 0.2600 - val_accuracy: 0.9450
Epoch 933/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0290 - accuracy: 0.9879 - val_loss: 0.2588 - val_accuracy: 0.9452
Epoch 934/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0290 - accuracy: 0.9878 - val_loss: 0.2643 - val_accuracy: 0.9451
Epoch 935/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0285 - accuracy: 0.9880 - val_loss: 0.2674 - val_accuracy: 0.9448
Epoch 

Epoch 987/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0286 - accuracy: 0.9881 - val_loss: 0.2745 - val_accuracy: 0.9433
Epoch 988/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0292 - accuracy: 0.9878 - val_loss: 0.2713 - val_accuracy: 0.9445
Epoch 989/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0284 - accuracy: 0.9883 - val_loss: 0.2686 - val_accuracy: 0.9435
Epoch 990/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0276 - accuracy: 0.9886 - val_loss: 0.2735 - val_accuracy: 0.9445
Epoch 991/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0278 - accuracy: 0.9882 - val_loss: 0.2737 - val_accuracy: 0.9447
Epoch 992/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0272 - accuracy: 0.9886 - val_loss: 0.2775 - val_accuracy: 0.9444
Epoch 993/1000
4/4 [==============================] - 13s 3s/step - loss: 0.0286 - accuracy: 0.9880 - val_loss: 0.2799 - val_accuracy: 0.9432
Epoch 

In [6]:
model.save("../models/cnn2")

np.save('../models/cnn2_train_loss.npy', np.array(history.history['loss']))
np.save('../models/cnn2_test_loss.npy', np.array(history.history['val_loss']))
np.save('../models/cnn2_train_acc.npy', np.array(history.history['accuracy']))
np.save('../models/cnn2_test_acc.npy', np.array(history.history['val_accuracy']))

INFO:tensorflow:Assets written to: ../models/cnn2/assets


## Model 2 Testing

In [8]:
Xtest, ytest = loadlocal_mnist(
    images_path='../emnist/emnist-letters-test-images-idx3-ubyte', 
    labels_path='../emnist/emnist-letters-test-labels-idx1-ubyte')

Xtest = Xtest.reshape(20800, 28, 28, 1)
Xtest = np.divide(Xtest, 255.)
ytest = ytest-1

In [9]:
yhat = model.predict(Xtest)
yhat = np.array(np.argmax(yhat, axis=1))
acc = accuracy_score(ytest, yhat)
f1 = f1_score(ytest, yhat, average='macro')
print(f'Accuracy is {acc}')
print(f'F1 score is {f1}')

Accuracy is 0.9430288461538462
F1 score is 0.9430571140935148


## Model 3 - Model 2 + FR2

In [ ]:
fr2_Xtrain = np.load(os.getcwd()+'/../feature_representations/feature_representation_2_train.npy')
fr2_Xtest = np.load(os.getcwd()+'/../feature_representations/feature_representation_2_test.npy')

tmp = X.flatten()
tmp_test = Xtest.flatten()

X = np.zeros(X.flatten().size + fr2_Xtrain.shape[0]) ???

In [ ]:
np.random.seed(4622)
tf.random.set_seed(4622)

img_input = layers.Input(shape=(28,28,1))

# Lambda layers to split
# Reshape layer for conv2d

x = layers.Conv2D(16, 3, activation='relu', padding='same')(img_input)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(32, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Dropout(0.4)(x)

x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)

x = layers.Dense(2048, activation='relu')(x)
# Dense layer somewhere in here
x = layers.Dropout(0.4)(x)

output = layers.Dense(26, activation='softmax')(x)

model = Model(img_input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(opt, loss)